In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import pandas as pd 

driver = webdriver.Chrome(ChromeDriverManager().install())
TOTAL_AMOUNT_TO_INVEST = 3000 # dollars
TOP_X_CRYPTOS = 50 # Thresholded up to 100



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/daniel/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/3d/vggd2dl96n937ldjr_8vgn240000gn/T/ipykernel_72979/2541299778.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
def get_soup(url): 
    # https://coderedirect.com/questions/215584/python-3-using-requests-does-not-get-the-full-content-of-a-web-page
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return soup 

def get_soup2(url): 
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup 

In [4]:
url_overrides = {
    "dai" : "multi-collateral-dai",
    "xdc-network" : "xinfin",
    '1inch-network' : '1inch',
    "gno" : "gnosis-gno",
    "pax-dollar": "paxos-standard"
}

In [5]:

url = "https://coinmarketcap.com/"
soup = get_soup(url)
table = soup.find('table', attrs={'class':'h7vnx2-2 czTsgW cmc-table'})
table_body = table.find('tbody')
url =  'https://coinmarketcap.com/'

results = []

rows = table_body.find_all('tr')
for row in rows[:min(100, TOP_X_CRYPTOS)]:
    # coin_item_symbol = row.find('p', class_ = "sc-1eb5slv-0 gGIpIK coin-item-symbol")
    coin_symbol = row.find('p', attrs={'class':"sc-1eb5slv-0 gGIpIK coin-item-symbol"}) 
    try: 
        if coin_symbol:
            coin_market_cap =row.find('span', attrs={'class': "sc-1ow4cwt-1 ieFnWP"})

        else: 
            coin_symbol = row.find('span', attrs={'class':"crypto-symbol"})
            url_symbol_text = row.find_all('span')[3].text.lower().replace(" ", "-").replace(".", "-")

            if url_symbol_text in url_overrides: 
                url_symbol_text = url_overrides[url_symbol_text]

            url = f'https://coinmarketcap.com/currencies/{url_symbol_text}/'
            market_cap_soup = get_soup2(url) 
            coin_market_cap = market_cap_soup.find_all('div', attrs={"class": "statsItemRight"})[0].find("div", attrs={"class": "statsValue"})

        coin_market_cap_int = int(coin_market_cap.text.replace(",", "").replace("$", ""))
    except: 
        print(f"Failed for token {coin_symbol}")

    results.append({'symbol': coin_symbol.text, 'market_cap': coin_market_cap_int})

In [8]:

investing_df = pd.DataFrame(results)

total_market_cap = investing_df['market_cap'].sum() 
investing_df['weight'] = investing_df['market_cap'] / total_market_cap
investing_df['amount_to_invest'] = investing_df['weight'] * TOTAL_AMOUNT_TO_INVEST
investing_df

,symbol,market_cap,weight,amount_to_invest
0,BTC,882701322370,0.425624,1276.872124
1,ETH,448167124905,0.216099,648.296422
2,BNB,86634369094,0.041774,125.320998
3,USDT,78351899921,0.037780,113.339988
4,SOL,52883591736,0.025500,76.498791
5,ADA,44860157152,0.021631,64.892487
6,USDC,42682059431,0.020581,61.741759
7,XRP,39740347930,0.019162,57.486424
8,LUNA,33390175779,0.016100,48.300579
9,DOT,29565789307,0.014256,42.768410
